In [ ]:
!apt install libcublas11
!pip install faster-whisper==0.10.0
!pip3 install transformers optimum accelerate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcublaslt11
The following NEW packages will be installed:
  libcublas11 libcublaslt11
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 226 MB of archives.
After this operation, 498 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 libcublaslt11 amd64 11.7.4.6~11.5.1-1ubuntu1 [148 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 libcublas11 amd64 11.7.4.6~11.5.1-1ubuntu1 [78.2 MB]
Fetched 226 MB in 9s (26.6 MB/s)
Selecting previously unselected package libcublaslt11:amd64.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../libcublaslt11_11.7.4.6~11.5.1-1ubuntu1_amd64.deb ...
Unpacking libcublaslt11:amd64 (11.7.4.6~11.5.1-1ubuntu1) ...
Selecting previously unselected package libcublas11:amd64.
P

In [ ]:
import os
from os import path
from google.colab import drive
drive.mount('/content/drive')

# files
src = "record.wav"
# convert wav to mp3

audio_path = os.path.join('/content/drive', 'MyDrive', src)

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3" #"Aspik101/distil-whisper-large-v3-pl"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=False
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
result = pipe(audio_path, generate_kwargs={"language": "polish"})
result

{'text': ' Dzień dobry. Dzień dobry. Proszę podać swoje imię, nazwisko i datę ubrudzenia. Iwan Klocuszek. 1 stycznia 1970. Dziękuję. Co pana do mnie sprowadza? Od przedwczoraj boli mnie gardło, a od wczoraj czuję się bardzo zmęczony i boli mnie głowa. Czy ma pan katar? Nie, ale mam zapchany nos. Jeszcze jakieś objawy może? Nie. Dobrze. A czy miał pan temperaturę? Nie mierzyłem. Rozumiem. To zaraz sprawdzimy. Proszę jeszcze powiedzieć, czy szczepił się pan na grypę lub COVID w tym roku? Dwa miesiące temu szczepiłem się na grypę, a na COVID ponad rok temu. Dobrze. Sprawdźmy temperaturę. 37 i 8. Proszę zdjąć maseczkę, otworzyć szeroko usta i powiedzieć a. A. Może pan założyć maseczkę. Czy brał pan już jakieś leki? Tylko prosz pan do ssania i buk profe. Dobrze. Dobrze. Teraz wypiszę panu ręki. Najważniejsze, żeby pan odpoczywał. Przepraszam.'}